### 필요한 모듈 가져오기

In [23]:
import pandas as pd
import numpy as np
import json
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import joblib

# XGBoost 사용
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder


### seed 값 고정

In [24]:
# 시드값이 고정이 되어 있지 않을 시 시드 값을 고정하여 모델을 학습 후 평가
## TODO: torch, tensorflow 등 사용 시에도 동일한 SEED 값 설정

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

### 데이터셋 불러오기

In [25]:
df = pd.read_csv('./train.csv')

In [26]:
df.head()

,src_ip,src_port,dst_ip,dst_port,protocol,num_packets,time_sequence,length_sequence,direction_sequence,label
0,150.242.169.98,51776,35.244.247.133,443,TCP,25,"[1625572258.480587, 1625572258.480702, 1625572...","[1470, 1470, 610, 40, 104, 210, 390, 454, 620,...","[-1, -1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, ...",38
1,172.16.30.110,49294,93.158.134.119,443,TCP,107,"[1607051927.782215, 1607051927.782324, 1607051...","[1450, 1450, 1450, 221, 52, 52, 52, 52, 132, 1...","[-1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -...",40
2,192.168.50.107,55750,38.121.72.166,443,TCP,213,"[1611731487.806609, 1611731487.806611, 1611731...","[1500, 636, 1151, 52, 52, 116, 52, 98, 95, 87,...","[-1, -1, -1, 1, 1, 1, -1, 1, 1, 1, -1, -1, -1,...",57
3,150.242.169.99,50485,59.108.138.228,443,TCP,12,"[1612662987.601396, 1612662987.601527, 1612662...","[1500, 1500, 315, 40, 120, 311, 311, 102, 40, ...","[-1, -1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1]",29
4,150.242.169.100,61777,109.244.244.164,443,TCP,18,"[1621999755.048606, 1621999755.048879, 1621999...","[1472, 1472, 1272, 1095, 40, 40, 40, 40, 120, ...","[-1, -1, -1, -1, 1, 1, 1, 1, 1, -1, -1, 1, -1,...",70


### 전처리

In [27]:
# JSON 형식의 문자열을 리스트로 변환

df['length_sequence'] = df['length_sequence'].apply(json.loads)
df['direction_sequence'] = df['direction_sequence'].apply(json.loads)
df['time_sequence'] = df['time_sequence'].apply(json.loads)

In [ ]:
# 입력 데이터에 5튜플에 대한 정보를 추가X
def preprocess(df):
    N = 30
    inputs = []
    
    for _, row in df.iterrows():
        length_seq = row['length_sequence']
        direction_seq = row['direction_sequence']
        time_seq = row['time_sequence']
        
        features = []
        
        # 패킷 길이 시퀀스
        padded_len = (length_seq[:N] + [0] * N)[:N]
        features.extend(padded_len)
        
        # 방향 시퀀스
        padded_dir = (direction_seq[:N] + [0] * N)[:N]
        features.extend(padded_dir)
        
        # 부호 있는 길이 시퀀스 (방향 * 길이)
        signed_len = [l * d for l, d in zip(length_seq, direction_seq)]
        padded_signed = (signed_len[:N] + [0] * N)[:N]
        features.extend(padded_signed)
        
        # 패킷 길이 통계
        features.append(len(length_seq))
        features.append(np.mean(length_seq))
        features.append(np.std(length_seq))
        features.append(np.min(length_seq))
        features.append(np.max(length_seq))
        features.append(np.median(length_seq))
        features.append(np.sum(length_seq))
        
        # Inter-arrival Time 통계
        if len(time_seq) > 1:
            iat = np.diff(time_seq)
            features.append(np.mean(iat))
            features.append(np.std(iat))
            features.append(np.min(iat))
            features.append(np.max(iat))
            features.append(np.median(iat))
        else:
            features.extend([0, 0, 0, 0, 0])
        
        # 방향별 통계
        out_lengths = [l for l, d in zip(length_seq, direction_seq) if d == 1]
        in_lengths = [l for l, d in zip(length_seq, direction_seq) if d == -1]
        
        features.append(len(out_lengths))
        features.append(np.sum(out_lengths) if out_lengths else 0)
        features.append(np.mean(out_lengths) if out_lengths else 0)
        features.append(np.std(out_lengths) if len(out_lengths) > 1 else 0)
        
        features.append(len(in_lengths))
        features.append(np.sum(in_lengths) if in_lengths else 0)
        features.append(np.mean(in_lengths) if in_lengths else 0)
        features.append(np.std(in_lengths) if len(in_lengths) > 1 else 0)
        
        total_pkts = len(length_seq)
        features.append(len(out_lengths) / total_pkts if total_pkts > 0 else 0)
        features.append(len(in_lengths) / total_pkts if total_pkts > 0 else 0)
        
        inputs.append(features)
    
    return inputs

In [29]:
# 전처리한 데이터를 열에 추가

df['input'] = preprocess(df)

### 학습, 검증 데이터 나누기

In [30]:
# Label Encoding (XGBoost는 0부터 시작하는 정수 레이블 필요)
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# 학습과 검증 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(
    df['input'], df['label_encoded'], test_size=0.2, random_state=SEED
)

# 모델 입력을 위해 리스트 형태로 변환
X_train = X_train.tolist()
y_train = y_train.tolist()

X_valid = X_valid.tolist()
y_valid = y_valid.tolist()

### 모델 학습

In [31]:
# XGBoost 모델 (강화된 설정)
model = XGBClassifier(
    n_estimators=200,          
    max_depth=12,              
    learning_rate=0.1,
    subsample=0.8,             
    colsample_bytree=0.8,      
    random_state=SEED,
    n_jobs=-1,
    verbosity=1
)
model.fit(X_train, y_train)

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


### 모델 검증

In [32]:
pred = model.predict(X_valid)
print(f"Accuracy: {accuracy_score(y_valid, pred):.4f}")
print(f"F1 Score: {f1_score(y_valid, pred, average='macro'):.4f}")

Accuracy: 0.8924
F1 Score: 0.8916


### 제출 파일 생성

In [33]:
test_df = pd.read_csv('./testcase.csv')

In [34]:
test_df.head()

,src_ip,src_port,dst_ip,dst_port,protocol,num_packets,time_sequence,length_sequence,direction_sequence
0,192.168.50.244,56819,192.0.77.32,443,TCP,27,"[1617178697.800111, 1617178697.800114, 1617178...","[1500, 248, 1500, 1500, 40, 40, 1040, 40, 160,...","[-1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, -1,..."
1,150.242.169.98,61627,99.86.202.24,443,TCP,21,"[1615726680.921434, 1615726680.921536, 1615726...","[1280, 1280, 1280, 1280, 1280, 772, 772, 52, 5...","[-1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1,..."
2,150.242.169.100,62098,182.254.59.150,443,TCP,24,"[1620715845.65485, 1620715845.654968, 16207158...","[1500, 1500, 1216, 1268, 41, 40, 120, 600, 438...","[-1, -1, -1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -..."
3,150.242.169.99,61645,184.25.254.123,443,TCP,49,"[1618907087.044121, 1618907087.044251, 1618907...","[1500, 1500, 1252, 1500, 52, 52, 52, 1349, 52,...","[-1, -1, -1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1..."
4,192.168.50.213,47564,59.111.181.35,443,TCP,64,"[1610526084.314479, 1610526084.314679, 1610526...","[1500, 1500, 781, 52, 52, 52, 132, 1500, 1500,...","[-1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1,..."


In [35]:
test_df['length_sequence'] = test_df['length_sequence'].apply(json.loads)
test_df['direction_sequence'] = test_df['direction_sequence'].apply(json.loads)
test_df['time_sequence'] = test_df['time_sequence'].apply(json.loads)

In [36]:
# 테스트 데이터 전처리 수행

test_df['input'] = preprocess(test_df)
X_test = test_df['input'].tolist()

In [37]:
# 모델 예측 (encoded -> 원래 레이블로 변환)

pred_test_encoded = model.predict(X_test)
pred_test = le.inverse_transform(pred_test_encoded)

In [38]:
# 결과물 저장
## TODO: 팀 번호에 맞게 수정해서 저장

TEAM = 11

###########################################################
# 아래 코드는 수정 X                                       #
# torch 또는 tensorflow로 모델 학습 시 모델 저장 함수만 수정 #
###########################################################
submit_df = pd.DataFrame({'label': pred_test})
submit_df.to_csv(f'./res/{TEAM}_submission.csv', index=False)

## TODO: torch 또는 tensorflow로 모델 학습 시 모델 저장 함수만 수정
joblib.dump(model, f'./res/{TEAM}_model.pkl')
joblib.dump(le, f'./res/{TEAM}_label_encoder.pkl')  # LabelEncoder도 저장
###########################################################
# 위 코드는 수정 X                                         #
# torch 또는 tensorflow로 모델 학습 시 모델 저장 함수만 수정 #
###########################################################

['./res/11_label_encoder.pkl']

In [39]:
total_count = len(df)
max_packets = df['num_packets'].max()
min_packets = df['num_packets'].min()
avg_packets = df['num_packets'].mean()

print(f"전체 데이터 수: {total_count}개")
print(f"패킷 수(Flow Length) - 최대: {max_packets}, 최소: {min_packets}, 평균: {avg_packets:.2f}")

df['mean_len'] = df['length_sequence'].apply(np.mean)
df['duration'] = df['time_sequence'].apply(lambda x: x[-1] - x[0] if len(x) > 1 else 0)

label_stats = df.groupby('label')[['num_packets', 'mean_len', 'duration']].mean()

top_len_label = label_stats['mean_len'].idxmax()
top_len_val = label_stats['mean_len'].max()
bot_len_label = label_stats['mean_len'].idxmin()
bot_len_val = label_stats['mean_len'].min()

print(f"평균 패킷 길이가 가장 긴 라벨: {top_len_label} (평균 {top_len_val:.2f} bytes)")
print(f"평균 패킷 길이가 가장 짧은 라벨: {bot_len_label} (평균 {bot_len_val:.2f} bytes)")




전체 데이터 수: 29552개
패킷 수(Flow Length) - 최대: 131182, 최소: 10, 평균: 168.33
평균 패킷 길이가 가장 긴 라벨: 21 (평균 765.89 bytes)
평균 패킷 길이가 가장 짧은 라벨: 17 (평균 170.71 bytes)
